## Что происходит в этом ноутбуке
Для начала: задача в этом ноутбуке -- разметить негативные отзывы с алиэкспресса.
На что именно покупатели с алика чаще всего жалуются. Видимо, это для команды приложения
"Шопинг" Т-банка.
1. Сначала я разметил данные через LLM-ку. В качестве модели я выбрал квантизованную
версию T-lite-it-1.0, так как это русскоязычная модель и так как при обучении могли использоваться данные
с приложения "Шопинг" Т-банка, поэтому она будет "легче" понимать тексты.
2. Затем я использовал snorkel для дополнительной разметки (это регулярки, решил, что без них здесь никак).
3. Предсказанные классы (и вероятности) от LLM я скомбинировал по формуле $0.6\cdot llm + 0.4\cdot snorkel$,
все, что ниже порога, отнес к "нет товара" (модель не уверена в товарах).
4. Далее нагенерировал синтетические примеры через ту же модель.
5. Затем я аугментировал примеры через замену слов синонимами, замену местами соседних слов и
замену случайных букв.
6. Наконец, я использовал QLoRA на той же модели для дообучения на примерах. Я добавил 8 токенов,
соответствующих каждой категории товаров, и дообучал модель предсказывать 1 токен товара.
На валидационной выборке удалось достичь качества 0.8.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/t-bank-nlp/NLP_case/submission_example.csv
/kaggle/input/t-bank-nlp/NLP_case/categories.txt
/kaggle/input/t-bank-nlp/NLP_case/README.md
/kaggle/input/t-bank-nlp/NLP_case/train.csv
/kaggle/input/t-bank-nlp/NLP_case/test.csv
/kaggle/input/train-predictions/train_augmented.parquet
/kaggle/input/train-predictions/synthetic_data.parquet
/kaggle/input/train-predictions/llm_predictions.parquet
/kaggle/input/train-predictions/train_with_labels.parquet
/kaggle/input/train-predictions/snorkel_predictions.parquet


In [2]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [11]:
import pandas as pd
df_train = pd.read_csv('/kaggle/input/t-bank-nlp/NLP_case/train.csv')

In [12]:
categories = pd.read_csv('/kaggle/input/t-bank-nlp/NLP_case/categories.txt')
categories.values.flatten()

array(['обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей',
       'украшения и аксессуары', 'электроника', 'нет товара'],
      dtype=object)

In [37]:
df_test = pd.read_csv('/kaggle/input/t-bank-nlp/NLP_case/test.csv')

In [14]:
CLASSES = ['обувь','одежда','посуда','текстиль','товары для детей',
           'украшения и аксессуары','электроника','нет товара']
IDX = {c:i for i,c in enumerate(CLASSES)}
ABSTAIN = -1

In [15]:
df_train[df_train['text'].str.contains(r'подошв')]

,text
853,Маленькие. Когда одеваешь рисунка почти нет. Б...


## Предобработка

In [16]:
df_train['text'] = df_train['text'].apply(lambda x: x.strip().lower().replace('\r', '\n'))

In [17]:
df_train

,text
0,"заказали 14.10.2017 , получили 25.10.2017 \n\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,все отлично!!!
3,"рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!низ рваный..деньги не вернули!открыла ...
...,...
1813,"спасибо,подошло по размеру.все понравилось в п..."
1814,"доставка быстрая, до саратова около 2 недель. ..."
1815,"на внешний вид шапка нормальная, на большой об..."
1816,за 4 месяца товар так и не дошел до покупателя.


In [38]:
df_test['text'] = df_test['text'].apply(lambda x: x.strip().lower().replace('\r', '\n'))

In [40]:
# df_train['text'] = df_train['text'].fillna("")
df_test['text'] = df_test['text'].fillna("")

In [42]:
df_test[df_test['text'] == '']

,text


## Загрузим модель, составим промт, дадим вероятности категории

### LLM для разметки

In [22]:
!pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 94.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [23]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.3 MB/s eta 0:00:00:00:0100:01


In [24]:
import torch, os, platform
print("torch:", torch.__version__, "CUDA:", torch.version.cuda)
print("GPU visible:", os.environ.get("CUDA_VISIBLE_DEVICES"))
print(platform.platform())


torch: 2.6.0+cu124 CUDA: 12.4
GPU visible: None
Linux-6.6.56+-x86_64-with-glibc2.35


In [25]:
import gc, torch
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [26]:
from transformers import BitsAndBytesConfig
DTYPE = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=DTYPE, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    quantization_config=bnb_cfg,
)
for param in model.parameters():
    param.requires_grad = False

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [29]:
classificator_prompt = '''
System: Ты — аккуратный классификатор отзывов маркетплейса по категориям товаров, к которым они относятся.
Доступные метки:
обувь,
одежда,
посуда,
текстиль,
товары для детей,
украшения и аксессуары,
электроника,
нет товара.

Правила:
- Если отзыв только про доставку/продавца/деньги/спор без товара → "нет товара".
- Если не ясно, какой товар — тоже "нет товара".
- Выбери ровно одну метку.

Ответь строго JSON одной строкой:
{"label": "<одна метка>", "confidence": 0..1, "reason": "<до 12 слов>"}

Примеры:

Отзыв: отличная блузка отслеживалась пришла за 10 дней.
Ответ: {"label": "одежда", "confidence": 1, "reason": "блузка относится к одежде"}

Отзыв: Лучшая покупка на алиэкспресс !!)))
Ответ: {"label": "нет товара", "confidence": 1, "reason": "нет никакого описания товара"}

Отзыв: На фото они матовые, а пришли глянцевые.
Ответ: {"label": "обувь", "confidence": 0.4, "reason": "матовость и глянцевость в множественном числе, вероятно, относится к обуви"}

Отзыв: Товар пришёл менее чем за месяц. На мой 44 заказала L пришла необъятных размеров  
Ответ: {"label": "одежда", "confidence": 0.9, "reason": "Размер L почти всегда относится к одежде"}
'''

In [30]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151665, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qw

In [31]:
df_train['text'].iloc[110]

'товар не пришел. деньги вернули. но продавец хороший и милый. просто, может быть, мне не повезло.'

In [33]:
messages = [
    {"role": "system", "content": classificator_prompt},
    {"role": "user", "content": df_train['text'].iloc[110]}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

{"label": "нет товара", "confidence": 1, "reason": "отзыв только о доставке и продавце"}


In [34]:
sum(p.numel() for p in model.parameters()) / 1024 / 1024 / 1024

4.051357746124268

In [28]:
model.device

device(type='cuda', index=0)

In [36]:
# Fast batched LLM labeling → adds "class_from_llm" and "llm_confidence" to df_train

import re, json, numpy as np
from tqdm.auto import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn.utils.rnn import pad_sequence

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # more efficient for decoder-only generation

model.eval()

# --- Helpers: prompt builder, robust JSON parser ---
def make_prompt(txt: str) -> str:
    messages = [
        {"role": "system", "content": classificator_prompt},   # <- you already defined this
        {"role": "user", "content": txt},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def extract_json(s: str):
    # Try a quick slice between first '{' and last '}' (common for chatty models)
    try:
        i, j = s.find("{"), s.rfind("}")
        if i != -1 and j != -1 and j > i:
            return json.loads(s[i:j+1])
    except Exception:
        pass
    # Fallback regex
    m = re.search(r"\{.*\}", s, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return None
    return None

def parse_output(text: str):
    obj = extract_json(text) or {}
    label = obj.get("label", None)
    conf  = obj.get("confidence", None)
    # normalize label
    if isinstance(label, str):
        label = label.strip().lower()
    if label not in CLASSES:
        # soft match (e.g., extra words)
        for c in CLASSES:
            if label and c in label:
                label = c; break
    if label not in CLASSES:
        label = "нет товара"   # safe fallback
    # coerce confidence
    try:
        conf = float(conf)
        if not (0.0 <= conf <= 1.0):
            conf = 0.5
    except Exception:
        conf = 0.5
    return label, conf

# --- Prepare data ---
df_train["text"] = df_train["text"].fillna("").astype(str)
texts = df_train["text"].tolist()

# --- Fast batched generation settings ---
BATCH_SIZE = 8            # ↓ if OOM, ↑ if room
MAX_INPUT_TOKENS = 512     # cap context for speed
MAX_NEW_TOKENS = 256        # JSON answer is short
GEN_KW = dict(
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,            # deterministic & fast
    temperature=0.0,
    top_p=1.0,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

pred_labels, pred_conf = [], []

with torch.inference_mode():
    for start in tqdm(range(0, len(texts), BATCH_SIZE), desc="LLM labeling"):
        batch_texts = texts[start:start+BATCH_SIZE]
        prompts = [make_prompt(t) for t in batch_texts]

        enc = tokenizer(
            prompts, return_tensors="pt",
            padding=True, truncation=True, max_length=MAX_INPUT_TOKENS
        ).to(model.device)

        input_lengths = enc["attention_mask"].sum(dim=1)  # prompt lengths per sample

        gen = model.generate(**enc, **GEN_KW)

        # Slice out only the newly generated tokens for each row
        new_tokens = [out_ids[in_len:] for out_ids, in_len in zip(gen, input_lengths)]

        # Pad to a rectangular tensor just for vectorized decode, then decode
        padded = pad_sequence(new_tokens, batch_first=True, padding_value=tokenizer.pad_token_id)
        out_texts = tokenizer.batch_decode(padded, skip_special_tokens=True)

        for out in out_texts:
            lbl, conf = parse_output(out)
            pred_labels.append(lbl)
            pred_conf.append(conf)

# --- Attach results to the DataFrame ---
df_train["class_from_llm"]   = pred_labels
df_train["llm_confidence"]   = pred_conf

# Optional quick sanity check
print(df_train["class_from_llm"].value_counts(dropna=False))
print(f"Mean confidence: {np.mean(pred_conf):.3f}")


LLM labeling:   0%|          | 0/228 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

class_from_llm
одежда                    1133
нет товара                 568
обувь                       55
текстиль                    28
украшения и аксессуары      19
посуда                      11
товары для детей             4
Name: count, dtype: int64
Mean confidence: 0.847


In [42]:
df_train.to_parquet('train_predictions.parquet')

## Snorkel для разметки

In [5]:
!pip install snorkel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [6]:
import re, numpy as np
from snorkel.labeling import labeling_function           # decorators still here
from snorkel.labeling import PandasLFApplier       # appliers moved to .apply
from snorkel.labeling.model import LabelModel            # LabelModel lives here
from snorkel.labeling.analysis import LFAnalysis         # analysis lives here

CLASSES = ['обувь','одежда','посуда','текстиль','товары для детей',
           'украшения и аксессуары','электроника','нет товара']
IDX = {c:i for i,c in enumerate(CLASSES)}
ABSTAIN = -1

# --- helpers
def has(text, rx): return re.search(rx, text, flags=re.I) is not None

# --- LFs (examples; extend!)
@labeling_function()
def lf_shoes_kw(x):
    if has(x.text, r'\b(ботинк|сапог|кроссовк|кеды|туфл|угг|босоножк|шл[её]панц|ботильон|валенк|)\w*\b'):
        return IDX['обувь']
    if has(x.text, r'\b(стельк|подошв|каблук|колодк)\w*\b'):
        return IDX['обувь']
    return ABSTAIN

@labeling_function()
def lf_clothes_kw(x):
    if has(x.text, r'(пуховик|куртк|ветровк|футболк|плать|джинс|рубашк|юбк|свитер|кофт|худи|термобель|колготк|пижам|халат|шорт|шарф|шапк|варежк|носки| S | M | L | XL | s | m | l | xl | м | л | с)\w*'):
        return IDX['одежда']
    if has(x.text, r'\b(рост|обхват|талия|бедра)\b') or has(x.text, r'(ткан|сшит)'):
        return IDX['одежда']
    return ABSTAIN

@labeling_function()
def lf_textile_kw(x):
    if has(x.text, r'(постельн|простын|наволочк|пододеяльник|полотенц|плед|покрывал|штор|тюл|скатерт|отрез ткан)\w*'):
        return IDX['текстиль']
    return ABSTAIN

@labeling_function()
def lf_tableware_kw(x):
    if has(x.text, r'(кружк|тарелк|кастрюл|сковород|ковш|форм[аы] для запекан|бокал|графин|ложк|вилк)\w*'):
        return IDX['посуда']
    return ABSTAIN

@labeling_function()
def lf_jewelry_kw(x):
    if has(x.text, r'(серьг|кольц|браслет|цепочк|кулон|чокер|брош|ремень|перчатк|очки|оправ)\w*'):
        return IDX['украшения и аксессуары']
    return ABSTAIN

@labeling_function()
def lf_electronics_kw(x):
    if has(x.text, r'(телефон|смартфон|наушник|колонк|ноутбук|телевизор|планшет|экран|камер|bluetooth|заряд|кабель|блок питания|ламп(а|очку)|пылесос|фен|триммер|блендер|миксер|электрочайн)\w*'):
        return IDX['электроника']
    return ABSTAIN

@labeling_function()
def lf_kids_kw(x):
    if has(x.text, r'(игрушк|кукл|конструктор|машинк|пазл|коляск|подгузник|бутылочк|соск|слюнявчик|погремушк|развива)\w*'):
        return IDX['товары для детей']
    return ABSTAIN

@labeling_function()
def lf_noproduct_service(x):
    if has(x.text, r'(доставк|курьер|трек|спор|возврат|деньг|продавц|магазин|приложени|оплат|скидк|купон)\w*'):
        return IDX['нет товара']
    if lf_shoes_kw(x) == ABSTAIN and lf_clothes_kw(x) == ABSTAIN and lf_textile_kw(x) == ABSTAIN \
      and lf_tableware_kw(x) == ABSTAIN and lf_jewelry_kw(x) == ABSTAIN and lf_electronics_kw(x) == ABSTAIN \
      and lf_kids_kw(x) == ABSTAIN:
        return IDX['нет товара']
    return ABSTAIN

# @labeling_function()
# def lf_too_short_generic(x):
#     if len(re.findall(r'\w+', x.text)) < 4 or has(x.text, r'товар|вернули'):
#         return IDX['нет товара']
#     return ABSTAIN


In [17]:
import pandas as pd
from snorkel.labeling import LFAnalysis

df_snorkel = pd.DataFrame({
    "text": df_train["text"].fillna("").astype(str)
})

lfs = [lf_shoes_kw, lf_clothes_kw, lf_textile_kw, lf_tableware_kw,
       lf_jewelry_kw, lf_electronics_kw, lf_kids_kw, lf_noproduct_service]
applier = PandasLFApplier(lfs)
L = applier.apply(df=df_snorkel)  # shape (N, num_LFs)

LFAnalysis(L=L, lfs=lfs).lf_summary()

label_model = LabelModel(cardinality=len(CLASSES), verbose=True)
label_model.fit(L_train=L, n_epochs=500, log_freq=50, seed=42)

probs = label_model.predict_proba(L)  # (N, 8)
lf_pred = probs.argmax(1)
lf_conf = probs.max(1)


100%|██████████| 500/500 [00:00<00:00, 629.10epoch/s]


In [18]:
probs

array([[0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.1354755 , 0.1354755 , 0.1186882 , ..., 0.11881734, 0.11868817,
        0.1354755 ],
       ...,
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.1354755 , 0.1354755 , 0.1186882 , ..., 0.11881734, 0.11868817,
        0.1354755 ],
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023]])

In [20]:
df_train['prediction_snorkel'] = lf_pred
df_train['snorkel_confidence'] = lf_conf

In [22]:
df_train["snorkel_probs"] = probs.tolist()

In [23]:
df_train

,text,prediction_snorkel,snorkel_confidence,snorkel_probs
0,"заказали 14.10.2017 , получили 25.10.2017 \n\n...",1,0.195202,"[0.1758284279266772, 0.19520214640769284, 0.09..."
1,"футболка хорошего качества,но футболка не как ...",1,0.195202,"[0.1758284279266772, 0.19520214640769284, 0.09..."
2,все отлично!!!,0,0.135475,"[0.1354754974159609, 0.1354754974159609, 0.118..."
3,"рисунок не очень чёткий, а ткань прозрачная, в...",1,0.195202,"[0.1758284279266772, 0.19520214640769284, 0.09..."
4,плохо!!!низ рваный..деньги не вернули!открыла ...,7,0.258021,"[0.19997417589112063, 0.20251091383449046, 0.0..."
...,...,...,...,...
1813,"спасибо,подошло по размеру.все понравилось в п...",0,0.135475,"[0.1354754974159609, 0.1354754974159609, 0.118..."
1814,"доставка быстрая, до саратова около 2 недель. ...",7,0.258021,"[0.19997417589112063, 0.20251091383449046, 0.0..."
1815,"на внешний вид шапка нормальная, на большой об...",1,0.195202,"[0.1758284279266772, 0.19520214640769284, 0.09..."
1816,за 4 месяца товар так и не дошел до покупателя.,0,0.135475,"[0.1354754974159609, 0.1354754974159609, 0.118..."


In [70]:
df_train.loc[1814]['text']

'доставка быстрая, до саратова около 2 недель. упакована качественно. сшита аккуратно, дефектов не обнаружила. ткань лёгкая, приятна к телу, немного (в меру) прозрачна. м на 44 размер.'

In [24]:
df_train.to_parquet('snorkel_predictions.parquet')

## Объединяем предсказания

In [26]:
import pandas as pd

df_train_snorkel = pd.read_parquet('/kaggle/input/train-predictions/snorkel_predictions.parquet')
df_train_llm = pd.read_parquet('/kaggle/input/train-predictions/llm_predictions.parquet')

In [38]:
p_snorkel = df_train_snorkel['snorkel_probs'].values
p_snorkel = np.array(list(p_snorkel))

In [39]:
p_snorkel

array([[0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.1354755 , 0.1354755 , 0.1186882 , ..., 0.11881734, 0.11868817,
        0.1354755 ],
       ...,
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023],
       [0.1354755 , 0.1354755 , 0.1186882 , ..., 0.11881734, 0.11868817,
        0.1354755 ],
       [0.17582843, 0.19520215, 0.09854587, ..., 0.09893634, 0.09854664,
        0.13585023]])

In [40]:
import numpy as np
import re

CLASSES = ['обувь','одежда','посуда','текстиль','товары для детей',
           'украшения и аксессуары','электроника','нет товара']
IDX = {c:i for i,c in enumerate(CLASSES)}
NO_PRODUCT = IDX['нет товара']

def build_llm_probs_from_label(lbls, confs, K=8):
    """
    If you only have LLM hard labels + a confidence in [0,1], build a prob vector:
    chosen = conf, the remaining mass is spread uniformly across others.
    """
    P = np.full((len(lbls), K), 0.0, dtype=np.float32)
    for i, (lab, conf) in enumerate(zip(lbls, confs)):
        conf = float(conf)
        conf = min(max(conf, 0.0), 1.0)
        base = (1.0 - conf) / (K - 1 + 1e-9)
        P[i, :] = base
        P[i, IDX[lab]] = conf
    return P

def fuse_probs(p_snorkel, p_llm, llm_conf=None, tau_low=0.60,
               alpha_min=0.20, alpha_max=0.80):
    """
    p_snorkel: (N,8) Snorkel LabelModel probs
    p_llm:     (N,8) LLM probs (from votes or build_llm_probs_from_label)
    llm_conf:  (N,) optional scalar confidence per row; if None -> max(p_llm, axis=1)
    tau_low:   if fused max prob < tau_low -> force 'нет товара'
    alpha_*:   LLM weight bounds for per-example mixing
    Returns: final_labels (N,), final_conf (N,), fused_probs (N,8)
    """
    eps = 1e-9
    # normalize & clip
    pS = np.clip(p_snorkel, eps, 1.0)
    pS /= pS.sum(1, keepdims=True)
    pL = np.clip(p_llm, eps, 1.0)
    pL /= pL.sum(1, keepdims=True)

    # per-example LLM weight from its confidence
    if llm_conf is None:
        llm_conf = pL.max(1)
    llm_conf = np.asarray(llm_conf, dtype=np.float32)
    alpha = np.clip(alpha_min + (alpha_max - alpha_min) * llm_conf, alpha_min, alpha_max)
    wL = alpha[:, None]
    wS = (1.0 - alpha)[:, None]

    # log-linear pooling (more robust than linear mixture)
    logp = wL * np.log(pL) + wS * np.log(pS)
    logp -= logp.max(1, keepdims=True)
    p = np.exp(logp)
    p /= p.sum(1, keepdims=True)

    y = p.argmax(1)
    conf = p.max(1)

    # Low-confidence fallback → 'нет товара'
    low = conf < float(tau_low)
    y[low] = NO_PRODUCT
    conf[low] = np.maximum(conf[low], tau_low)  # optional: floor for readability

    return y, conf, p

# Optional tiny guard: if text is service-only and fused conf is not high, force 'нет товара'
SERVICE_RX = re.compile(r'(доставк|курьер|трек|спор|возврат|деньг|продавц|приложени|оплат)', re.I)
def postprocess_service(texts, y, conf, p, tau_service=0.70):
    for i, t in enumerate(texts):
        if y[i] != NO_PRODUCT and conf[i] < tau_service and SERVICE_RX.search(t or ""):
            y[i] = NO_PRODUCT
    return y, conf


In [41]:
p_llm = build_llm_probs_from_label(df_train_llm["class_from_llm"], df_train_llm["llm_confidence"], K=len(CLASSES))
y, conf, p = fuse_probs(p_snorkel, p_llm, llm_conf=df_train_llm["llm_confidence"], tau_low=0.50)
final_labels = [CLASSES[i] for i in y]


In [42]:
from collections import Counter
Counter(final_labels)

Counter({'одежда': 1130,
         'нет товара': 598,
         'обувь': 38,
         'украшения и аксессуары': 18,
         'текстиль': 28,
         'товары для детей': 3,
         'посуда': 3})

In [43]:
df_train['label'] = final_labels

In [44]:
df_train.to_parquet('train_with_labels.parquet')

## Генерируем синтетику

In [20]:
!pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [21]:
CLASSES

['обувь',
 'одежда',
 'посуда',
 'текстиль',
 'товары для детей',
 'украшения и аксессуары',
 'электроника',
 'нет товара']

In [22]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 27.2 MB/s eta 0:00:00:00:0100:01


In [43]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import torch

DTYPE = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
torch.manual_seed(42)

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"": 0},   
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    quantization_config=bnb_cfg,
)
for param in model.parameters():
    param.requires_grad = False

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
df_train_with_labels = pd.read_parquet('/kaggle/input/train-predictions/train_with_labels.parquet')

In [28]:
df_train_with_labels = df_train_with_labels[['text', 'label']]

In [44]:
# --- CONFIG ---
import os, re, json, math, random, gc
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from torch.nn.utils.rnn import pad_sequence

np.random.seed(42)
random.seed(42)

# Your 8 classes (labels already exist in df_train['label'])
CLASSES = ['обувь','одежда','посуда','текстиль','товары для детей',
           'украшения и аксессуары','электроника','нет товара']

assert {'text','label'}.issubset(df_train_with_labels.columns), "df_train must have columns: text, label"

# Make sure tokenizer/model are set for batched decoding
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151665, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qw

In [32]:
# --- 1) Mine top in-corpus n-grams per class (hints) ---
def top_ngrams_for_class(df, label, topk=40, ngram_range=(1,2), min_df=2):
    texts = df.loc[df['label']==label, 'text'].fillna("").astype(str).tolist()
    if len(texts) == 0:
        return []
    vec = CountVectorizer(ngram_range=ngram_range, min_df=min_df, lowercase=True,
                          token_pattern=r"[A-Za-zА-Яа-я0-9ёЁ]+")
    X = vec.fit_transform(texts)
    freqs = np.asarray(X.sum(axis=0)).ravel()
    vocab = np.array(vec.get_feature_names_out())
    # Drop overly generic tokens
    keep = [i for i,t in enumerate(vocab) if len(t) >= 3]
    idx = np.argsort(freqs[keep])[::-1][:topk]
    return vocab[keep][idx].tolist()

HINTS = {c: top_ngrams_for_class(df_train_with_labels, c, topk=40) for c in CLASSES}


In [33]:
HINTS

{'обувь': ['размер',
  'очень',
  'все',
  'рост',
  'качество',
  'носки',
  'подошёл',
  'товар',
  'было',
  'раз',
  'отлично',
  'после',
  'пришёл',
  'пришли',
  'после первой',
  'первой носки',
  'первой',
  'размеру',
  'на фото',
  'меня',
  'как',
  'доставка',
  'быстро',
  'больше',
  'по размеру',
  'хорошее',
  'шел',
  'что',
  'хорошо',
  'фото',
  'цвет',
  'месяц',
  'материал на',
  'материал',
  'качество хорошее',
  'хорошо но',
  'заказывала',
  'заказ',
  'за 2',
  'есть'],
 'одежда': ['очень',
  'размер',
  'качество',
  'как',
  'что',
  'доставка',
  'ткань',
  'платье',
  'соответствует',
  'все',
  'так',
  'спасибо',
  'цвет',
  'рост',
  'заказала',
  'быстро',
  'пришла',
  'фото',
  'синтетика',
  'хорошо',
  'это',
  'заказывала',
  'продавец',
  'рекомендую',
  'быстрая',
  'деньги',
  'мне',
  'как на',
  'не соответствует',
  'на фото',
  'нитки',
  'товар',
  'материал',
  'рукава',
  'размеру',
  'супер',
  'хорошая',
  'только',
  'даже',
  'тор

In [77]:
PROMPT_EXAMPLES = {
    'обувь': [
        "1. слишком долгая доставка, пахнут резиной",
        "2. маленькие. когда одеваешь рисунка почти нет. белая подошва."
    ],
    'одежда': [
        "1. цвет черный а не темно синий , обидно. шапка хорошая."
        "2. на 2 размера меньше пришел , плохо прошит , полоски стерлись на следующий день , синтетик и не теплый",
        "3. заказали 14.10.2017 , получили 25.10.2017 \n\nна мой размер 42, широкий как мешок. надо было все таки размер  s заказать. \n\nпо поводу качества хороший пуховик. \n\nмех натуральный , съемный. \n\nбуду продавать .",
        "4. футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой м, заказала л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.",
        "5. рисунок не очень чёткий, а ткань прозрачная, видно нижнее бельё"
    ],
    'посуда': [
        "1. для тех, кто пьёт напитки литрами и не хочет постоянно отвлекаться на то, чтобы их делать.",
        "2. вздумалось мне заказать на алиэкспресс стальную мисочку для лапши объёмом 1 000 мл. и вот заказ прибыл. нужно доставать и проверять."
    ],
    'текстиль': [
        "1. приятная, нежная ткань. хороший комплект. только цвет не такой сочный, как на картинке.",
        "2. мое спасение! для ёрзающих во сне)))",
        "3. теплый велюровый комплект постельного белья + дополнительные наволочки."
    ],
    'товары для детей': [
        "1. шапку покупала ребёнку из за пампона (он чёрный)так вот он совсем не держится на шапке!!!продавцу написала об этом но он игнорит!",
        "2. для ребенка старше года бесполезная. для детей помладше или тех, что спят как статуи, возможно, сработает."
    ],
    'украшения и аксессуары': [
        "1. фигня, нитки торчат, хотела носить как пояс-баска, размер оказался огромный",
        "2. эти серьги я покупала примерно в начале 2022 года. они потерялись, не отслеживались на территории рф - я была вынуждена открыть диспут и получить деньги обратно. ",
        "3. самые классные цепочки"
    ],
    'электроника': [
        "1. так себе прибор для путешествий",
        "2. недорогой спидометр, работает нормально",
        "3. купил светодиодную лампу, прошло три дня, очень хорошо светит когда сидишь за столом,но иногда лампа перегревается но все норм или сидишь за компьютером, думаю ещё купить лампу другу на день рождение и себе вторую лампу, всем доволен, всем рекомендую"
    ],
    'нет товара': [
        "1. все отлично!!!",
        "2. доставка до спб 35 дней.",
        "3. по началу всё отслеживалось, но в конце выяснилось, что пришло в балашиху, а заказывала в саратов! ждала, но заказ так и не пришёл, защита закончилась",
        "4. заказ так и не пришёл, открыла спор, продавец его не принял, просил закрыть спор и подождать, пришлось привлекать алиэкспресс, возврат денег одобрили, деньги пришли.",
    ]
}

# --- 2) Prompt templates (Russian) ---
GEN_SYSTEM = (
    "Ты генерируешь реалистичные отзывы для маркетплейса на русском языке. "
    "Следуй классу и качеству формулировок корпуса. Без брендов, без ссылок. "
    "Разнообразь стиль (полож./нейтр./негат.), избегай повторов. "
    "Выводи строго JSONL: по одному объекту на строку. "
    "Каждый объект: {\"text\": \"<отзыв без переносов>\", \"label\": \"<класс>\"}. "
    "Не используй фигурные скобки внутри текста, без markdown и подсказок."
    "Примеры реальных отзывов:"
    )

def make_gen_prompt(category: str, n_items: int, hints: list[str]) -> str:
    # Compose concise hints (up to ~12 tokens/phrases)
    sampled = ", ".join(random.sample(hints, k=min(12, max(3, len(hints))))) if hints else ""
    user = (
        f"Сгенерируй {n_items} реалистичных отзывов категории «{category}». "
        "Длина до 40 слов; варьируй стиль (положительный/нейтральный/негативный). "
        "Если категория «нет товара», делай отзывы про доставку/продавца/деньги без упоминания конкретного товара. "
        f"Подсказки корпуса: {sampled}\n"
        "Верни строго JSONL (одна строка — один объект) вида:\n"
        "{\"text\":\"...\", \"label\":\"" + category + "\"}"
    )
    messages = [
        {"role": "system", "content": GEN_SYSTEM + '\n'.join(PROMPT_EXAMPLES[category])},
        {"role": "user", "content": user},
    ]
    # chat→string
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# --- 3) Robust JSONL extraction ---
def extract_jsonl_objects(text: str):
    # Strip code fences if any
    text = re.sub(r"```(?:json)?\s*|\s*```", "", text, flags=re.I)
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    objs = []
    for ln in lines:
        # quick filter: looks like a JSON object
        if not (ln.startswith("{") and ln.endswith("}")):
            continue
        try:
            obj = json.loads(ln)
            if isinstance(obj, dict) and "text" in obj and "label" in obj:
                objs.append(obj)
        except Exception:
            continue
    return objs

# --- 4) Batched generation core ---
def generate_batch(prompts: list[str],
                   max_input_tokens=1000,
                   max_new_tokens=1500,
                   do_sample=True,
                   temperature=0.8,
                   top_p=0.9,
                   repetition_penalty=1.05):
    enc = tokenizer(
        prompts, return_tensors="pt",
        padding=True, truncation=True, max_length=max_input_tokens
    ).to(model.device)

    with torch.inference_mode():
        gen = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=True,
        )

    # Slice new tokens only
    input_lens = enc["attention_mask"].sum(dim=1)
    new_tokens = [out_ids[in_len:] for out_ids, in_len in zip(gen, input_lens)]
    # Pad & decode
    padded = pad_sequence(new_tokens, batch_first=True, padding_value=tokenizer.pad_token_id)
    out_texts = tokenizer.batch_decode(padded, skip_special_tokens=True)
    return out_texts


In [66]:
prompts = []
prompts.append(make_gen_prompt(category='электроника', n_items=10, hints=HINTS.get('электроника', [])))
prompts

['<|im_start|>system\nТы генерируешь реалистичные отзывы для маркетплейса на русском языке. Следуй классу и качеству формулировок корпуса. Без брендов, без ссылок. Разнообразь стиль (полож./нейтр./негат.), избегай повторов. Выводи строго JSONL: по одному объекту на строку. Каждый объект: {"text": "<отзыв без переносов>", "label": "<класс>"}. Не используй фигурные скобки внутри текста, без markdown и подсказок.Примеры реальных отзывов:1. супер прибор для путешествий\n2. недорогой спидометр, работает нормально\n3. купил светодиодную лампу, прошло три дня, очень хорошо светит когда сидишь за столом,но иногда лампа перегревается но все норм или сидишь за компьютером, думаю ещё купить лампу другу на день рождение и себе вторую лампу, всем доволен, всем рекомендую<|im_end|>\n<|im_start|>user\nСгенерируй 10 реалистичных отзывов категории «электроника». Длина до 80 слов; варьируй стиль (положительный/нейтральный/негативный). Если категория «нет товара», делай отзывы про доставку/продавца/деньг

In [67]:
outs = generate_batch(prompts)

In [68]:
outs

['```jsonl\n{"text":"У меня появился новый смартфон, и он превзошел все мои ожидания. Качество камеры просто потрясающее, а аккумулятор держит заряд целый день. Очень доволен покупкой.", "label":"электроника"}\n{"text":"Приобрел ноутбук для работы и учебы. Он работает стабильно, но немного шумит при нагрузке. В целом, хорошая покупка для тех, кто ищет универсальное устройство.", "label":"электроника"}\n{"text":"Этот Bluetooth-динамик отлично звучит, но батарея разряжается слишком быстро. Надеюсь, это временное явление, так как качество звука действительно впечатляет.", "label":"электроника"}\n{"text":"Купил умные часы, они удобны в использовании, но периодически зависают. Надеюсь, обновление прошивки решит проблему.", "label":"электроника"}\n{"text":"Светильник с LED-технологией светит ярко, но потребляет много электроэнергии. Для экономии стоит рассмотреть другие варианты.", "label":"электроника"}\n{"text":"Приобрел умные колонки, которые стали центром моей домашней развлекательной си

In [78]:

# --- 5) Light QA filters for realism ---
RX_CYR = re.compile(r"[А-Яа-яЁё]")
def qa_keep(obj: dict, category: str):
    txt = obj.get("text","").strip()
    lbl = str(obj.get("label","")).strip().lower()
    if lbl != category:
        return False
    # one line, reasonable length, Russian chars present
    if ("\n" in txt) or (len(txt.split()) < 12) or (len(txt.split()) > 90):
        return False
    if RX_CYR.search(txt) is None:
        return False
    # avoid trivial boilerplate
    if len(set(txt.lower().split())) < 8:
        return False
    return True

# --- 6) Driver: generate N per class ---
def generate_synthetics(df, target_per_class=250,
                        prompts_per_call=2,
                        items_per_prompt=20):
    rows = []
    seen = set()  # to dedupe by text
    for category in CLASSES:
        needed = target_per_class
        pbar = tqdm(total=needed, desc=f"Gen {category}")
        while needed > 0:
            # Build a small batch of prompts
            prompts = []
            for _ in range(prompts_per_call):
                k = min(items_per_prompt, needed)
                prompts.append(make_gen_prompt(category, n_items=k, hints=HINTS.get(category, [])))
            # Generate
            outs = generate_batch(prompts)
            # Parse & collect
            got = 0
            for out in outs:
                objs = extract_jsonl_objects(out)
                for obj in objs:
                    if qa_keep(obj, category):
                        t = obj["text"].strip()
                        if t not in seen:
                            rows.append({"text": t, "label": category, "provenance": "llm_gen"})
                            seen.add(t)
                            got += 1
                            pbar.update(1)
                            needed -= 1
                            if needed <= 0:
                                break
                if needed <= 0:
                    break
            # Small GC to keep mem tidy on Kaggle
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        pbar.close()
    return pd.DataFrame(rows, columns=["text","label","provenance"])

# --- 7) Run generation ---
df_synth = generate_synthetics(df_train, target_per_class=50,
                               prompts_per_call=2,   # try 2–4 depending on VRAM
                               items_per_prompt=5)  # 20 JSON objects per prompt
print(df_synth.head(), "\nCounts:\n", df_synth['label'].value_counts())

Gen обувь:   0%|          | 0/50 [00:00<?, ?it/s]

Gen одежда:   0%|          | 0/50 [00:00<?, ?it/s]

Gen посуда:   0%|          | 0/50 [00:00<?, ?it/s]

Gen текстиль:   0%|          | 0/50 [00:00<?, ?it/s]

Gen товары для детей:   0%|          | 0/50 [00:00<?, ?it/s]

Gen украшения и аксессуары:   0%|          | 0/50 [00:00<?, ?it/s]

Gen электроника:   0%|          | 0/50 [00:00<?, ?it/s]

Gen нет товара:   0%|          | 0/50 [00:00<?, ?it/s]

                                                text  label provenance
0  Моя первая покупка у этого продавца, и я остал...  обувь    llm_gen
1  Получил заказ быстро, но обувь маломерит на од...  обувь    llm_gen
2  Обувь отлично сидит по размеру, удобная и стил...  обувь    llm_gen
3  На фото обувь выглядит лучше, чем в реальности...  обувь    llm_gen
4  Пришла обувь в целости и сохранности, но мне о...  обувь    llm_gen 
Counts:
 label
обувь                     50
одежда                    50
посуда                    50
текстиль                  50
товары для детей          50
украшения и аксессуары    50
электроника               50
нет товара                50
Name: count, dtype: int64


In [79]:
df_synth.to_parquet('synthetic_data.parquet')

In [83]:
df_synth[df_synth['label'] == 'товары для детей']['text'].values

array(['Очень довольны покупкой! Шапка идеально подходит по размеру, держится на голове ребёнка без проблем. Рекомендую!',
       'Нейтрально. Покупали комбинезон для малыша. Подходит по возрасту, но хотелось бы более долговечный материал.',
       'Разочарован покупкой. Несмотря на заявленный вес, кресло для автокресла оказалось слишком легким и неустойчивым. Не рекомендую.',
       'Приобрели мягкую игрушку для годовалого сына. Вес и размер соответствуют описанию, но ребенок быстро потерял интерес.',
       'Купили новый весовой набор для измерения роста и веса. Все отлично работает, точность высокая.',
       'Ребёнок очень любит новую книжку с интерактивными элементами. Но срок службы аккумулятора оказался коротким.',
       'Покупка детского стульчика для кормления оказалась разочарованием. Он неустойчивый и легко опрокидывается.',
       'Шапка отлично держится на голове моего малыша! Материал приятный на ощупь и теплый. Рекомендую!',
       'Коляски этого бренда удобны для наших

In [74]:
df_synth[df_synth['label'] == 'одежда']['text'].values

array(['Недавно заказала платье, но оно пришло с дефектом - одна нить вылезла. Обидно, так как продавец уверял в высоком качестве. Платье какое-то скромное, не соответствует описанию. Вернула и жду возврата денег.',
       'Футболка идеально соответствует описанию: качество отличное, сшита аккуратно, цвет как на фото. Рекомендую!'],
      dtype=object)

In [40]:
print("hf_device_map:", getattr(model, "hf_device_map", "n/a"))
try:
    from accelerate import cpu_offload
except Exception:
    pass


hf_device_map: {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 1, 'model.layers.8': 1, 'model.layers.9': 1, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}


In [37]:
sum_cpu = sum(1 for _,p in model.named_parameters() if p.device.type != "cuda")
print("Layers on CPU:", sum_cpu)  # should be 0

Layers on CPU: 0


## Создаем аугментации

In [4]:
!pip install ruwordnet nltk pymorphy3

In [5]:
!ruwordnet download

downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.4/ruwordnet-2021.db


In [7]:
import pandas as pd

df_train_synth = pd.read_parquet('/kaggle/input/train-predictions/synthetic_data.parquet')
df_train_with_labels = pd.read_parquet('/kaggle/input/train-predictions/train_with_labels.parquet')
df_train_full = pd.concat([df_train_with_labels, df_train_synth])

In [12]:
# === Text Augmentation Toolkit (RU/EN friendly) ===
import re, random, math
import numpy as np
import pandas as pd
from tqdm import tqdm
from ruwordnet import RuWordNet
from pymorphy3 import MorphAnalyzer
import nltk
from nltk.corpus import wordnet as wn


_ = wn.ensure_loaded()
morph = MorphAnalyzer()
rwn = RuWordNet()

random.seed(42)
np.random.seed(42)

# --- Alphabets for char noise ---
RU_LOWER = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя")
RU_UPPER = [c.upper() for c in RU_LOWER]
EN_LOWER = list("abcdefghijklmnopqrstuvwxyz")
EN_UPPER = [c.upper() for c in EN_LOWER]

def _rand_other(char_list, cur):
    if len(char_list) == 1: return cur
    c = cur
    while c == cur:
        c = random.choice(char_list)
    return c

def char_noise(text: str, p_char: float = 0.02) -> str:
    """Replace each RU/EN letter with a random same-script, same-case letter with prob p_char."""
    out = []
    for ch in text:
        r = random.random()
        if r >= p_char:
            out.append(ch); continue
        if ch in RU_LOWER:
            out.append(_rand_other(RU_LOWER, ch))
        elif ch in RU_UPPER:
            out.append(_rand_other(RU_UPPER, ch))
        elif ch in EN_LOWER:
            out.append(_rand_other(EN_LOWER, ch))
        elif ch in EN_UPPER:
            out.append(_rand_other(EN_UPPER, ch))
        else:
            out.append(ch)  # don’t touch digits/punct/space
    return "".join(out)

RX_CYR = re.compile(r"[А-Яа-яЁё]")

def _transfer_case(src: str, dst: str) -> str:
    if src.isupper(): return dst.upper()
    if src.istitle(): return dst.capitalize()
    return dst

def ru_synonyms(word: str, max_k: int = 5) -> list[str]:
    """Russian synonyms via RuWordNet (if available). Returns lemmas; tries to inflect back."""
    if rwn is None or morph is None: return []
    p = morph.parse(word)[0]
    lemma = p.normal_form
    syns = set()
    try:
        # Collect lemmas from all synsets where lemma appears
        for ss in rwn.get_synsets(lemma):
            for lit in ss.literals:
                cand = lit.text.lower()
                if cand != lemma:
                    syns.add(cand)
    except Exception:
        return []
    syns = list(syns)[:max_k]
    # try to inflect to the same grammemes
    out = []
    for s in syns:
        sp = morph.parse(s)[0]
        # attempt to match grammemes intersection
        gramm = {g for g in p.tag.grammemes if g in sp.tag.grammemes}
        try:
            inf = sp.inflect(gramm) if gramm else None
            out.append((inf.word if inf else s))
        except Exception:
            out.append(s)
    return out

def en_synonyms(word: str, max_k: int = 5) -> list[str]:
    if wn is None: return []
    wlow = word.lower()
    syns = set()
    try:
        for ss in wn.synsets(wlow, lang='eng'):
            for l in ss.lemmas(lang='eng'):
                cand = l.name().replace("_"," ").lower()
                if cand != wlow:
                    syns.add(cand)
    except Exception:
        return []
    return list(syns)[:max_k]

def get_synonyms(word: str) -> list[str]:
    """Auto-detect RU/EN and fetch synonyms; may return []."""
    if RX_CYR.search(word):
        return ru_synonyms(word)
    else:
        return en_synonyms(word)

WORD_RX = re.compile(r"[A-Za-zА-Яа-яЁё]+")

def synonym_replace(text: str, p_word: float = 0.12) -> str:
    """
    With prob p_word per token, replace by a synonym (if available).
    Keeps punctuation/spacing; preserves case shape of the original token.
    """
    tokens = re.findall(r"\w+|[^\w\s]+|\s+", text, flags=re.UNICODE)
    out = []
    for tok in tokens:
        if WORD_RX.fullmatch(tok) and random.random() < p_word:
            syns = get_synonyms(tok)
            syns = [s for s in syns if s.lower() != tok.lower()]
            if syns:
                repl = random.choice(syns)
                out.append(_transfer_case(tok, repl))
                continue
        out.append(tok)
    return "".join(out)

def swap_adjacent_words(text: str, p_swap: float = 0.08) -> str:
    """
    Swap adjacent word tokens with probability p_swap (non-overlapping).
    Non-word tokens (punct/space) keep their positions relative to words.
    """
    # Split into words and separators, then operate on word indices
    parts = re.findall(r"[A-Za-zА-Яа-яЁё]+|[^A-Za-zА-Яа-яЁё]+", text)
    # Identify word indices
    idx = [i for i, p in enumerate(parts) if WORD_RX.fullmatch(p)]
    i = 0
    while i < len(idx) - 1:
        if random.random() < p_swap:
            a, b = idx[i], idx[i+1]
            parts[a], parts[b] = parts[b], parts[a]
            i += 2  # skip next to avoid overlapping swaps
        else:
            i += 1
    return "".join(parts)

# --- Top-level driver to build augmented DataFrame ---
def augment_dataframe(df: pd.DataFrame,
                      p_char: float = 0.03,
                      p_syn: float = 0.2,
                      p_swap: float = 0.2,
                      which=("char_noise","synonym","swap"),
                      keep_original=True,
                      repeat=1) -> pd.DataFrame:
    """
    Apply selected augmentations to each row of df[['text','label']].
    Returns a new dataframe with columns: text, label, augmentation.
    """
    assert {'text','label'}.issubset(df.columns)
    rows = []
    for i, (txt, lab) in tqdm(enumerate(df[['text','label']].itertuples(index=False))):
        for _ in range(repeat):
            text = "" if pd.isna(txt) else str(txt)
            if "swap" in which:
                text = swap_adjacent_words(text, p_swap)
            if "synonym" in which:
                text = synonym_replace(text, p_syn)
            if "char_noise" in which:
                text = char_noise(text, p_char)
            rows.append({"text": text, "label": lab, "augmentation":"true"})
        if keep_original:
            rows.append({"text": txt, "label": lab, "augmentation":"false"})
    return pd.DataFrame(rows, columns=["text","label","augmentation"])

In [15]:
df_train_full = df_train_full[['text', 'label']]

In [16]:
df_augmented = augment_dataframe(df_train_full, p_char=0.03, p_syn=0.15, p_swap=0.2,
                                 which=("char_noise","synonym","swap"),
                                 keep_original=True, repeat=3)
print(df_augmented.head())
print(df_augmented['augmentation'].value_counts())

2218it [00:13, 163.25it/s]

                                                text   label augmentation
0  заказали 14.10.2017 , получили 25.10.2017 \n\n...  одежда         true
1  голучили 14.10.2017 , заказали 25.10.2017 \n\n...  одежда         true
2  получили 14.10.2017 , заказали 25.10.2017 \n\n...  одежда         true
3  заказали 14.10.2017 , получили 25.10.2017 \n\n...  одежда        false
4  футболка хорошего качества,но фугболка не как ...  одежда         true
augmentation
true     6654
false    2218
Name: count, dtype: int64


In [18]:
df_augmented.head(10)['text'].values

array(['заказали 14.10.2017 , получили 25.10.2017 \n\nмой на рязмер 42, широкий как мешок. надо было все таки размер  s заказать. \n\nпо поводу хороший качества пуховик. \n\nмех съемный , натуральный. \n\nпродавать буду .',
       'голучили 14.10.2017 , заказали 25.10.2017 \n\nмой на размер 42, широкий как мешок. надо было все таки размер  s заказать. \n\nпо поводу качества пуховик хоьосий. \n\nнатуральный мех , буду. \n\nсъемный продавать .',
       'получили 14.10.2017 , заказали 25.10.2017 \n\nна мой широкий 42, ркзмюр кщк мешок. надо все было таки s  размср заказать. \n\nпо поводу хороший качестоа пуховик. \n\nнатуральный мех , съемный. \n\nпродавать буду .',
       'заказали 14.10.2017 , получили 25.10.2017 \n\nна мой размер 42, широкий как мешок. надо было все таки размер  s заказать. \n\nпо поводу качества хороший пуховик. \n\nмех натуральный , съемный. \n\nбуду продавать .',
       'футболка хорошего качества,но фугболка не как девушек для и женщин,как а мужчину на. она очень с

In [19]:
df_augmented.to_parquet('train_augmented.parquet')

### QLoRA

In [3]:
!pip install -U "transformers==4.44.2" "trl==0.9.6" "peft==0.11.1" "accelerate==0.33.0" "bitsandbytes==0.44.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 91.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB

In [4]:
# !pip install transformers bitsandbytes

In [5]:
import transformers

transformers.__version__

'4.44.2'

In [6]:
import trl
trl.__version__

'0.9.6'

In [7]:
import pandas as pd

df_augmented = pd.read_parquet('/kaggle/input/train-predictions/train_augmented.parquet')

In [8]:
import numpy as np
import pandas as pd

def stratified_fraction(df: pd.DataFrame, label_col: str = "label",
                        frac: float = 0.5, random_state: int = 42) -> pd.DataFrame:
    """
    Return a stratified subsample with size ≈ frac * len(df), preserving label fractions.
    Ensures the total is exactly round(frac * N) via fractional rounding.
    """
    assert 0 < frac < 1, "frac must be in (0,1)"
    assert label_col in df.columns, f"'{label_col}' column not found"

    counts = df[label_col].value_counts()
    desired = counts * frac
    target_total = int(round(len(df) * frac))

    floors = np.floor(desired).astype(int)
    remainder = target_total - floors.sum()

    targets = floors.copy()

    if remainder > 0:
        # Give +1 to classes with largest fractional part
        fractional = (desired - floors).sort_values(ascending=False)
        for lab in fractional.index[:remainder]:
            targets[lab] += 1
    elif remainder < 0:
        # Remove 1 from classes with smallest fractional part (but not below 0)
        fractional = (desired - floors).sort_values(ascending=True)
        removed = 0
        for lab in fractional.index:
            if removed == -remainder:
                break
            if targets[lab] > 0:
                targets[lab] -= 1
                removed += 1

    # Sample per class
    parts = []
    for lab, n in targets.items():
        if n <= 0:
            continue
        grp = df[df[label_col] == lab]
        # If a class is tiny and rounding asked for more than available (shouldn't happen), clip:
        n = min(n, len(grp))
        parts.append(grp.sample(n=n, random_state=random_state))

    df_out = pd.concat(parts, axis=0).sample(frac=1.0, random_state=random_state).reset_index(drop=True)

    # Sanity checks (optional)
    print("Original size:", len(df), "→ New size:", len(df_out))
    print("Original dist:\n", (df[label_col].value_counts(normalize=True)).sort_index())
    print("New dist:\n", (df_out[label_col].value_counts(normalize=True)).sort_index())

    return df_out

df_augmented = stratified_fraction(df_augmented, label_col="label", frac=0.5, random_state=42)

Original size: 8872 → New size: 4436
Original dist:
 label
нет товара                0.292155
обувь                     0.039675
одежда                    0.532011
посуда                    0.023895
текстиль                  0.035167
товары для детей          0.023895
украшения и аксессуары    0.030658
электроника               0.022543
Name: proportion, dtype: float64
New dist:
 label
нет товара                0.292155
обувь                     0.039675
одежда                    0.532011
посуда                    0.023895
текстиль                  0.035167
товары для детей          0.023895
украшения и аксессуары    0.030658
электроника               0.022543
Name: proportion, dtype: float64


In [9]:
# ===================== QLoRA single-token classifier (Weighted F1) =====================
# Requirements: transformers>=4.43, peft>=0.11, trl>=0.9, bitsandbytes>=0.43, scikit-learn
import os, re, json, random, gc
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          LogitsProcessor)
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, SFTConfig
from datasets import Dataset, DatasetDict

2025-09-17 04:21:56.448854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758082916.648834      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758082916.713495      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
# -------------------- Setup & constants --------------------
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cuda.matmul.allow_tf32 = True

BASE_MODEL = "t-tech/T-lite-it-1.0"  # your T-Lite checkpoint
CLASSES = ['обувь','одежда','посуда','текстиль','товары для детей',
           'украшения и аксессуары','электроника','нет товара']
IDX = {c:i for i,c in enumerate(CLASSES)}

# Ensure df_augmented has the right columns
assert {'text','label'}.issubset(df_augmented.columns), "df_augmented must have 'text' and 'label'"

# -------------------- Train/val split (stratified) --------------------
df_augmented = df_augmented.sample(frac=1.0, random_state=42).reset_index(drop=True)
train_df, val_df = train_test_split(
    df_augmented[['text','label']].dropna(),
    test_size=0.15, random_state=42, stratify=df_augmented['label']
)
train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

print("Train size:", len(train_df), "Eval size:", len(val_df))
print("Train class counts:\n", train_df['label'].value_counts())

Train size: 3770 Eval size: 666
Train class counts:
 label
одежда                    2006
нет товара                1101
обувь                      149
текстиль                   133
украшения и аксессуары     116
посуда                      90
товары для детей            90
электроника                 85
Name: count, dtype: int64


In [11]:
!pip -q install sentencepiece

In [12]:
# -------------------- Tokenizer & label tokens --------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"  # better for decoder-only

# One special token per class; model will learn to generate exactly this single token
label_to_token = {c: f"<cat_{i}>" for i, c in enumerate(CLASSES)}
token_to_label = {v: k for k, v in label_to_token.items()}
specials = list(label_to_token.values())
tokenizer.add_special_tokens({"additional_special_tokens": specials})

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

8

In [13]:
train_df

,text,label
0,"Заказ был отправлен месяц но, назад до сих пор...",нет товара
1,цвет совсем не такой как был заказан,нет товара
2,качество на троечку,нет товара
3,"шорты пбдошли размером на м кг 48 вес, рост 1...",одежда
4,"получить Ржидал красисую миску, но пришла она ...",посуда
...,...,...
3765,товар не пишел. никакого общенияс продавцом не...,нет товара
3766,заказ пнишел через месяц до окутска. классный ...,нет товара
3767,ну такое. в целом неплохо. на тдоечку за ьакие...,нет товара
3768,кяута классная. пришла без размера. заказывала...,одежда


In [14]:
# -------------------- Prompt & tokenization --------------------
SYSTEM_INSTR = (
    "Ты — точный классификатор отзывов маркетплейса. Выбери ровно один ярлык "
    "из списка. Если отзыв про доставку/продавца/деньги/спор без явного товара — метка «нет товара». "
    "Отвечай только одним служебным токеном класса."
)

def build_chat_prompt(text: str) -> str:
    labels_str = ", ".join([f'{label_to_token[c]}({c})' for c in CLASSES])
    messages = [
        {"role": "system", "content": SYSTEM_INSTR},
        {"role": "user", "content": f"Отзыв:\n{text}\n\nМетки: {labels_str}"},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

def tokenize_row(ex):
    # Format as: [prompt][answer_token]; mask all prompt tokens, supervise only the last token
    txt = "" if ex["text"] is None else str(ex["text"])
    ytok = label_to_token[ex["label"]]
    prompt = build_chat_prompt(txt)
    full   = prompt + ytok
    enc = tokenizer(full, truncation=True, max_length=512)   # no padding here
    L = len(enc["input_ids"])
    enc["labels"] = [-100]*(L-1) + [enc["input_ids"][-1]]    # 1 label per token, last is supervised
    # ensure plain Python lists of ints (no numpy/ragged objs)
    for k in ("input_ids","attention_mask","labels"):
        enc[k] = list(map(int, enc[k]))
    return enc

In [15]:
from transformers import DataCollatorForLanguageModeling

train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

train_ds = Dataset.from_pandas(train_df[["text","label"]])
val_ds   = Dataset.from_pandas(val_df[["text","label"]])

train_tok = train_ds.map(tokenize_row, remove_columns=train_ds.column_names, desc="Tokenize train")
val_tok   = val_ds.map(tokenize_row,   remove_columns=val_ds.column_names,   desc="Tokenize val")


Tokenize train:   0%|          | 0/3770 [00:00<?, ? examples/s]

Tokenize val:   0%|          | 0/666 [00:00<?, ? examples/s]

In [16]:
ds = DatasetDict(train=train_tok, validation=val_tok)

In [17]:
# sanity: lengths match and are 1-D
def _check(ds, n=5):
    for i in range(min(n, len(ds))):
        L = len(ds[i]["input_ids"])
        assert len(ds[i]["labels"]) == L and isinstance(ds[i]["labels"][-1], int)
_check(train_tok)
_check(val_tok)

 # custom collator that pads inputs AND labels together
def collate_fn(batch):
     pad_id = tokenizer.pad_token_id
     max_len = max(len(x["input_ids"]) for x in batch)
     input_ids, attn_mask, labels = [], [], []
     for ex in batch:
         L = len(ex["input_ids"]); pad_len = max_len - L
         input_ids.append(torch.tensor(ex["input_ids"] + [pad_id]*pad_len, dtype=torch.long))
         attn_mask.append(torch.tensor(ex["attention_mask"] + [0]*pad_len, dtype=torch.long))
         labels.append(torch.tensor(ex["labels"] + [-100]*pad_len, dtype=torch.long))  # <- pad labels
     return {
         "input_ids": torch.stack(input_ids, 0),
         "attention_mask": torch.stack(attn_mask, 0),
         "labels": torch.stack(labels, 0),
     }

In [18]:
!pip install -U "triton==3.0.0"
   # installs a recent compatible Triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires triton==3.2.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.0.0 which is incompatible.


In [19]:
# import gc, torch, sys

# # 1) Delete big refs you created earlier
# for name in ["trainer","model","base","tok_fast","tok_slow","enc","gen","ds","train_tok","val_tok"]:
#     if name in globals():
#         try:
#             del globals()[name]
#         except Exception:
#             pass

# # 2) Collect garbage & free CUDA cache
# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.ipc_collect()

# # 3) Quick memory check (optional)
# if torch.cuda.is_available():
#     used = torch.cuda.memory_allocated()/1024**3
#     reserved = torch.cuda.memory_reserved()/1024**3
#     print(f"After cleanup — used: {used:.2f} GB, reserved: {reserved:.2f} GB")


In [20]:
from peft import prepare_model_for_kbit_training

# -------------------- Load 4-bit base & wrap with LoRA --------------------
DTYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True,
                             bnb_4bit_compute_dtype=DTYPE,
                             bnb_4bit_use_double_quant=True,
                             bnb_4bit_quant_type="nf4")

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg,
    torch_dtype=DTYPE,
    device_map={"": 0} if torch.cuda.is_available() else "auto",
    low_cpu_mem_usage=True
)
# Resize embeddings to include new class tokens
base.resize_token_embeddings(len(tokenizer))
base = prepare_model_for_kbit_training(base)

peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],  # common for LLaMA/Mistral-like
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(base, peft_cfg)
model.config.use_cache = False

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [21]:
# Optional but helpful diagnostics
model.print_trainable_parameters()  # should show a nonzero % trainable
print("Any trainable params?", any(p.requires_grad for p in model.parameters()))

trainable params: 40,370,176 || all params: 7,653,184,000 || trainable%: 0.5275
Any trainable params? True


In [22]:
for i in range(len(ds["train"])):
    ex = ds["train"][i]
    if not (isinstance(ex["labels"], list) and len(ex["labels"]) == len(ex["input_ids"])
            and all(isinstance(x, int) for x in ex["labels"])):
        print("Bad row:", i, type(ex["labels"]), len(ex["labels"]), len(ex["input_ids"]))
        break

In [23]:
from trl import SFTTrainer, SFTConfig
sft_cfg = SFTConfig(
    output_dir="qlora_cls",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=20,
    save_steps=500,
    bf16=(DTYPE==torch.bfloat16),
    fp16=(DTYPE==torch.float16),
    gradient_checkpointing=False,
    optim="adamw_torch",
    max_seq_length=384,
    report_to=[],
)
trainer = SFTTrainer(
    model=model,
    args=sft_cfg,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    tokenizer=tokenizer,
    data_collator=collate_fn,   # use custom collator that pads labels with -100
    packing=False,
)
trainer.train()


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,6.713400
40,0.518800


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=58, training_loss=2.605840510335462, metrics={'train_runtime': 11694.7493, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.005, 'total_flos': 7.283218325354496e+16, 'train_loss': 2.605840510335462, 'epoch': 1.9661016949152543})

In [24]:

# -------------------- Constrained decoding for evaluation --------------------
class LabelOnlyProcessor(LogitsProcessor):
    def __init__(self, allowed_ids, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.allowed = torch.tensor(allowed_ids, device=device)
    def __call__(self, input_ids, scores):
        # Mask everything except class tokens
        mask = torch.full_like(scores, float("-inf"))
        mask.index_fill_(1, self.allowed, 0.0)
        return scores + mask

allowed_ids = tokenizer.convert_tokens_to_ids(list(label_to_token.values()))
processor = LabelOnlyProcessor(allowed_ids)

def predict_labels(texts, model, batch_size=32, max_input_tokens=512):
    preds = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        prompts = [build_chat_prompt(t) for t in batch]
        enc = tokenizer(prompts, return_tensors="pt",
                        padding=True, truncation=True, max_length=max_input_tokens).to(model.device)
        with torch.inference_mode():
            gen = model.generate(**enc,
                                 max_new_tokens=1,           # we want exactly one label token
                                 do_sample=False,            # greedy
                                 logits_processor=[processor],
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.pad_token_id,
                                 use_cache=True)
        # take only new tokens after the prompt
        new = gen[:, enc["input_ids"].shape[1]:]
        toks = tokenizer.batch_decode(new, skip_special_tokens=False)
        # map token -> class (first token)
        for t in toks:
            tok = t.strip().split()[0] if t.strip() else ""
            preds.append(token_to_label.get(tok, "нет товара"))
    return preds


In [25]:
# -------------------- Weighted F1 on validation --------------------
y_true = val_df["label"].tolist()
y_pred = predict_labels(val_df["text"].tolist(), batch_size=32)

print(classification_report(y_true, y_pred, labels=CLASSES, digits=4))
print("Weighted F1:", f1_score(y_true, y_pred, average="weighted"))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation res

                        precision    recall  f1-score   support

                 обувь     0.6250    0.5556    0.5882        27
                одежда     0.9404    0.7571    0.8388       354
                посуда     1.0000    1.0000    1.0000        16
              текстиль     1.0000    0.4783    0.6471        23
      товары для детей     1.0000    0.6250    0.7692        16
украшения и аксессуары     0.9167    0.5500    0.6875        20
           электроника     1.0000    0.8000    0.8889        15
            нет товара     0.6250    0.9487    0.7536       195

              accuracy                         0.7928       666
             macro avg     0.8884    0.7143    0.7717       666
          weighted avg     0.8408    0.7928    0.7959       666

Weighted F1: 0.7958554186829252


In [26]:
import os, json

OUT_DIR = "qlora_cls_adapter"

# 1) Save the PEFT adapter weights/config (small)
model.save_pretrained(OUT_DIR)          # or: trainer.save_model(OUT_DIR)

# 2) Save tokenizer (so special class tokens are preserved)
tokenizer.save_pretrained(OUT_DIR)

# 3) Save your label↔token mapping for inference
with open(os.path.join(OUT_DIR, "label_tokens.json"), "w", encoding="utf-8") as f:
    json.dump({"label_to_token": label_to_token}, f, ensure_ascii=False, indent=2)

# 4) (Optional) Save training state/args for reproducibility
trainer.save_state()
with open(os.path.join(OUT_DIR, "training_args.json"), "w") as f:
    f.write(trainer.args.to_json_string())


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [34]:
!zip -r qlora_cls.zip /kaggle/working/qlora_cls
!zip -r qlora_cls_adapter /kaggle/working/qlora_cls_adapter

  adding: kaggle/working/qlora_cls/ (stored 0%)
  adding: kaggle/working/qlora_cls/trainer_state.json (deflated 59%)
  adding: kaggle/working/qlora_cls/checkpoint-58/ (stored 0%)
  adding: kaggle/working/qlora_cls/checkpoint-58/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/qlora_cls/checkpoint-58/vocab.json (deflated 69%)
  adding: kaggle/working/qlora_cls/checkpoint-58/adapter_model.safetensors^C



zip error: Interrupted (aborting)
  adding: kaggle/working/qlora_cls_adapter/ (stored 0%)
  adding: kaggle/working/qlora_cls_adapter/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/qlora_cls_adapter/vocab.json (deflated 69%)
  adding: kaggle/working/qlora_cls_adapter/adapter_model.safetensors^C



zip error: Interrupted (aborting)


In [30]:
model_qlora = model

In [ ]:
import json, torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, BitsAndBytesConfig

DTYPE = torch.float16 if torch.cuda.get_device_capability()[0] < 8 else torch.bfloat16
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=DTYPE,
                             bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

ADAPTER_DIR = "qlora_cls_adapter"

# AutoPeftModel will read base_model_name_or_path from the adapter config and load base+adapter
model = AutoPeftModelForCausalLM.from_pretrained(
    ADAPTER_DIR, quantization_config=bnb_cfg, device_map="auto"
).eval()

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR, use_fast=False)
with open(f"{ADAPTER_DIR}/label_tokens.json", "r", encoding="utf-8") as f:
    label_to_token = json.load(f)["label_to_token"]
token_to_label = {v:k for k,v in label_to_token.items()}


## Разметка тестового набора

In [35]:
from transformers import BitsAndBytesConfig
DTYPE = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=DTYPE, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

for param in base.parameters():
    param.requires_grad = False


In [45]:
model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [36]:
classificator_prompt = '''
System: Ты — аккуратный классификатор отзывов маркетплейса по категориям товаров, к которым они относятся.
Доступные метки:
обувь,
одежда,
посуда,
текстиль,
товары для детей,
украшения и аксессуары,
электроника,
нет товара.

Правила:
- Если отзыв только про доставку/продавца/деньги/спор без товара → "нет товара".
- Если не ясно, какой товар — тоже "нет товара".
- Выбери ровно одну метку.

Ответь строго JSON одной строкой:
{"label": "<одна метка>", "confidence": 0..1, "reason": "<до 12 слов>"}

Примеры:

Отзыв: отличная блузка отслеживалась пришла за 10 дней.
Ответ: {"label": "одежда", "confidence": 1, "reason": "блузка относится к одежде"}

Отзыв: Лучшая покупка на алиэкспресс !!)))
Ответ: {"label": "нет товара", "confidence": 1, "reason": "нет никакого описания товара"}

Отзыв: На фото они матовые, а пришли глянцевые.
Ответ: {"label": "обувь", "confidence": 0.4, "reason": "матовость и глянцевость в множественном числе, вероятно, относится к обуви"}

Отзыв: Товар пришёл менее чем за месяц. На мой 44 заказала L пришла необъятных размеров  
Ответ: {"label": "одежда", "confidence": 0.9, "reason": "Размер L почти всегда относится к одежде"}
'''

In [46]:
import re, json, numpy as np
from tqdm.auto import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

# --- tokenizer hygiene ---
# make sure both pad_token_id and pad_token are set (Qwen2 slow tokenizer can be picky)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base.eval()

# ---- helpers (as before) ----
def make_prompt(txt: str) -> str:
    messages = [
        {"role": "system", "content": classificator_prompt},
        {"role": "user", "content": txt},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def extract_json(s: str):
    try:
        i, j = s.find("{"), s.rfind("}")
        if i != -1 and j != -1 and j > i:
            return json.loads(s[i:j+1])
    except Exception:
        pass
    m = re.search(r"\{.*\}", s, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return None
    return None

def parse_output(text: str):
    obj = extract_json(text) or {}
    label = obj.get("label", None)
    conf  = obj.get("confidence", None)
    if isinstance(label, str):
        label = label.strip().lower()
    if label not in CLASSES:
        for c in CLASSES:
            if label and c in label:
                label = c; break
    if label not in CLASSES:
        label = "нет товара"
    try:
        conf = float(conf)
        if not (0.0 <= conf <= 1.0):
            conf = 0.5
    except Exception:
        conf = 0.5
    return label, conf

# --- data ---
df_test["text"] = df_test["text"].fillna("").astype(str)
texts = df_test["text"].tolist()

# --- generation settings (clean: no sampling-only args) ---
BATCH_SIZE = 8
MAX_INPUT_TOKENS = 512
MAX_NEW_TOKENS = 24   # JSON is short; keep small for speed

GEN_KW = dict(
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,                      # greedy
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    use_cache=True
)

pred_labels, pred_conf = [], []

# safe per-row decode to avoid None tokens from padded matrix
def safe_decode(ids_tensor):
    # filter out any PAD / negative ids before decode
    ids = ids_tensor.tolist()
    ids = [i for i in ids if isinstance(i, int) and i >= 0 and i != tokenizer.pad_token_id]
    if not ids:
        return ""
    return tokenizer.decode(ids, skip_special_tokens=True)

with torch.inference_mode():
    for start in tqdm(range(0, len(texts), BATCH_SIZE), desc="LLM labeling"):
        batch_texts = texts[start:start+BATCH_SIZE]
        prompts = [make_prompt(t) for t in batch_texts]

        enc = tokenizer(
            prompts, return_tensors="pt",
            padding=True, truncation=True, max_length=MAX_INPUT_TOKENS
        ).to(base.device)

        input_lengths = enc["attention_mask"].sum(dim=1)

        gen = base.generate(**enc, **GEN_KW)

        # slice only the newly generated tokens
        new_tokens = [out_ids[in_len:] for out_ids, in_len in zip(gen, input_lengths)]

        # per-sample decode (robust for Qwen2 slow tokenizer)
        out_texts = [safe_decode(t) for t in new_tokens]

        for out in out_texts:
            lbl, conf = parse_output(out)
            pred_labels.append(lbl)
            pred_conf.append(conf)

df_test["class_from_llm"] = pred_labels
df_test["llm_confidence"] = pred_conf

print(df_test["class_from_llm"].value_counts(dropna=False))
print(f"Mean confidence: {np.mean(pred_conf):.3f}")


LLM labeling:   0%|          | 0/910 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


TypeError: sequence item 0: expected str instance, NoneType found

In [ ]:
df_test.to_parquet("test_markup_llm")

In [ ]:
L_test = applier.apply(df=df_test)  # shape (N, num_LFs)
test_probs = label_model.predict_proba(L_test)  # (N, 8)
lf_pred = test_probs.argmax(1)
lf_conf = test_probs.max(1)

df_test['prediction_snorkel'] = lf_pred
df_test['snorkel_confidence'] = lf_conf
df_test["snorkel_probs"] = test_probs.tolist()

df_test.to_parquet('test_markup_snorkel.parquet')

In [ ]:
p_snorkel = df_test['snorkel_probs'].values
p_snorkel = np.array(list(p_snorkel))

In [ ]:
p_llm = build_llm_probs_from_label(df_test["class_from_llm"], df_test["llm_confidence"], K=len(CLASSES))
y, conf, p = fuse_probs(p_snorkel, p_llm, llm_conf=df_test["llm_confidence"], tau_low=0.50)
final_labels = [CLASSES[i] for i in y]

In [ ]:
df_test['label'] = final_labels

In [ ]:
# -------------------- Weighted F1 on validation --------------------
y_true = df_test["label"].tolist()
y_pred = predict_labels(df_test["text"].tolist(), model=model_qlora, batch_size=32)

print(classification_report(y_true, y_pred, labels=CLASSES, digits=4))
print("Weighted F1:", f1_score(y_true, y_pred, average="weighted"))

### PEFT

In [ ]:
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=DTYPE, device_map="auto")
base.resize_token_embeddings(len(tokenizer))

pt_cfg = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=64,  # 20-100; more = more capacity
    tokenizer_name_or_path=BASE_MODEL,
)
model = get_peft_model(base, pt_cfg)

sft_cfg = SFTConfig(
    output_dir="prompt_tune_cls",
    num_train_epochs=3,
    per_device_train_batch_size=4, gradient_accumulation_steps=8,
    learning_rate=5e-4, warmup_ratio=0.05,
    logging_steps=20, save_steps=500, bf16=(DTYPE==torch.bfloat16),
    gradient_checkpointing=True
)
trainer = SFTTrainer(model=model, train_dataset=ds["train"], eval_dataset=ds["validation"],
                     tokenizer=tokenizer, args=sft_cfg)
trainer.train()


### Fine-Tuning

In [ ]:
SMALL_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # example
tokenizer = AutoTokenizer.from_pretrained(SMALL_MODEL, use_fast=True)
tokenizer.add_special_tokens({"additional_special_tokens": specials})
model = AutoModelForCausalLM.from_pretrained(SMALL_MODEL, torch_dtype=DTYPE, device_map="auto")
model.resize_token_embeddings(len(tokenizer))
model.gradient_checkpointing_enable()

sft_cfg = SFTConfig(
    output_dir="fullft_cls_small",
    num_train_epochs=3,
    per_device_train_batch_size=4, gradient_accumulation_steps=8,
    learning_rate=1e-5, weight_decay=0.01, warmup_ratio=0.03,
    bf16=(DTYPE==torch.bfloat16), logging_steps=20, save_steps=500
)
trainer = SFTTrainer(model=model, train_dataset=ds["train"], eval_dataset=ds["validation"],
                     tokenizer=tokenizer, args=sft_cfg)
trainer.train()
